# Objective: check splits.

In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import anndata as ad
import scanpy as sc
from sklearn.model_selection import StratifiedKFold

In [2]:
import sys
print (sys.executable)
print(sys.version)

/projects/pfenninggroup/jemmie/mathys_hg_singlecell/miniconda3/envs/r42mamba/bin/python
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]


In [6]:
base_dir ="/projects/pfenninggroup/singleCell/Ruzicka_snRNA_Seq"
data_dir = base_dir+ "/data/"

### create splits first.


In [7]:
ps=ad.read_h5ad(base_dir+ "/data/22_08_12_seurat_raw_FILTERED_neurons_only_controls_only_pseudobulk.h5ad")
ps

AnnData object with n_obs × n_vars = 69 × 17658
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ID', 'Age', 'Phenotype'
    var: '_index', 'features'

In [8]:
meta =pd.read_csv(base_dir +"/data/"+"Ruzicka_metadata.csv")
meta.head

/tmp/ipykernel_114777/1061557879.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  meta =pd.read_csv(base_dir +"/data/"+"Ruzicka_metadata.csv")


<bound method NDFrame.head of                      index                  Id  sizeFactors   Cohort     ID  \
0       AAACCCAAGAACGCGT-1  AAACCCAAGAACGCGT-1     0.600758   McLean    SZ2   
1       AAACCCAAGTACTCGT-1  AAACCCAAGTACTCGT-1     1.148219   McLean    SZ2   
2       AAACCCACAATGAACA-1  AAACCCACAATGAACA-1     1.255586   McLean   CON3   
3       AAACCCAGTATAGCTC-1  AAACCCAGTATAGCTC-1     0.577726   McLean   CON1   
4       AAACCCATCATGAAAG-1  AAACCCATCATGAAAG-1     1.461431   McLean   CON2   
...                    ...                 ...          ...      ...    ...   
468722       B120405_C2314       B120405_C2314     0.361440  MtSinai  CON75   
468723       B120405_C2318       B120405_C2318     0.153178  MtSinai  CON75   
468724       B120405_C2319       B120405_C2319     0.166351  MtSinai  CON75   
468725       B120405_C2320       B120405_C2320     0.163879  MtSinai  CON75   
468726       B120405_C2325       B120405_C2325     0.157229  MtSinai  CON75   

        Internal_ID  

In [9]:
meta=meta[meta["Phenotype"]== "CON"]
meta.shape

(262713, 33)

In [10]:
meta=meta[meta["ID"].isin(set(ps.obs['ID']))]
meta.shape

(260832, 33)

In [11]:
meta=meta[["ID", "Gender", "Age"]].drop_duplicates()
meta.shape

(69, 3)

In [12]:
# make age a binned variable
meta['Age_Bin'] = pd.qcut(meta['Age'],q=4)
meta['Age_Bin'].value_counts()

(23.999, 58.0]    19
(69.0, 81.0]      17
(81.0, 94.0]      17
(58.0, 69.0]      16
Name: Age_Bin, dtype: int64

In [25]:
meta['age_gender']=  meta["Age_Bin"].astype(str)+meta["Gender"]
meta['age_gender'].value_counts()

(23.999, 58.0]Male      14
(81.0, 94.0]Male        11
(58.0, 69.0]Female       9
(69.0, 81.0]Female       9
(69.0, 81.0]Male         8
(58.0, 69.0]Male         7
(81.0, 94.0]Female       6
(23.999, 58.0]Female     5
Name: age_gender, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
meta.index = meta["ID"]
df_train, df_test = train_test_split(meta, test_size=0.15, stratify=meta[["age_gender"]], random_state=42)

In [35]:
#make a new age-gender bin.
# df_train['Age_Bin'].value_counts()
# df_train['age_gender']=  df_train["Age_Bin"].astype(str)+df_train["Gender"]
df_train['age_gender'].value_counts()

(23.999, 58.0]Male      12
(81.0, 94.0]Male         9
(69.0, 81.0]Female       8
(58.0, 69.0]Female       7
(69.0, 81.0]Male         7
(58.0, 69.0]Male         6
(81.0, 94.0]Female       5
(23.999, 58.0]Female     4
Name: age_gender, dtype: int64

In [36]:
#tst['age_gender']= df_test["Age_Bin"].astype(str)+df_test["Gender"]
df_test['age_gender'].value_counts()

(23.999, 58.0]Male      2
(81.0, 94.0]Male        2
(58.0, 69.0]Female      2
(58.0, 69.0]Male        1
(23.999, 58.0]Female    1
(81.0, 94.0]Female      1
(69.0, 81.0]Male        1
(69.0, 81.0]Female      1
Name: age_gender, dtype: int64

In [37]:
df_test.shape

(11, 5)

In [56]:
# save a copy
df_test.to_csv(base_dir+"/data/22_11_02_df_test.csv")
#df_train.to_csv(base_dir+"/data/22_11_02_df_train.csv")

In [32]:
# temp1 = pd.read_csv(base_dir+"/data/22_08_16_df_train.csv")
# np.unique(temp1['ID'])

array(['CON1', 'CON11', 'CON12', 'CON13', 'CON15', 'CON16', 'CON17',
       'CON18', 'CON19', 'CON2', 'CON20', 'CON22', 'CON24', 'CON25',
       'CON29', 'CON3', 'CON30', 'CON31', 'CON33', 'CON34', 'CON35',
       'CON36', 'CON37', 'CON38', 'CON4', 'CON40', 'CON41', 'CON42',
       'CON44', 'CON45', 'CON46', 'CON49', 'CON51', 'CON53', 'CON54',
       'CON55', 'CON56', 'CON57', 'CON58', 'CON59', 'CON6', 'CON60',
       'CON63', 'CON65', 'CON66', 'CON67', 'CON68', 'CON69', 'CON70',
       'CON71', 'CON72', 'CON73', 'CON75', 'CON8', 'CON9'], dtype=object)

In [38]:
np.unique(df_train['ID'])

array(['CON1', 'CON10', 'CON11', 'CON12', 'CON13', 'CON14', 'CON15',
       'CON16', 'CON17', 'CON18', 'CON19', 'CON2', 'CON21', 'CON23',
       'CON24', 'CON25', 'CON27', 'CON28', 'CON29', 'CON3', 'CON30',
       'CON31', 'CON33', 'CON34', 'CON35', 'CON36', 'CON37', 'CON4',
       'CON40', 'CON41', 'CON42', 'CON44', 'CON45', 'CON49', 'CON5',
       'CON51', 'CON52', 'CON53', 'CON54', 'CON55', 'CON57', 'CON59',
       'CON60', 'CON61', 'CON62', 'CON63', 'CON64', 'CON66', 'CON67',
       'CON68', 'CON69', 'CON70', 'CON71', 'CON72', 'CON73', 'CON74',
       'CON8', 'CON9'], dtype=object)

### Add a validation split. 

In [40]:
train, df_val = train_test_split(df_train, test_size=0.15, stratify=df_train[["age_gender"]], random_state=42)
train.shape

(49, 5)

In [41]:
train['age_gender'].value_counts()

(23.999, 58.0]Male      10
(81.0, 94.0]Male         8
(69.0, 81.0]Female       7
(69.0, 81.0]Male         6
(58.0, 69.0]Female       6
(58.0, 69.0]Male         5
(81.0, 94.0]Female       4
(23.999, 58.0]Female     3
Name: age_gender, dtype: int64

In [43]:
train.to_csv(base_dir+"/data/22_11_02_df_train.csv")
df_val.to_csv(base_dir+"/data/22_11_02_df_val.csv")

#### Check all IDs are in the final file.

In [58]:
import functools
len(set().union(*[set(train['ID'].values), 
                  set(df_val['ID'].values), 
                  set(df_test['ID'].values)])) # yes. It's all 55.

69

Now, let's visualize our folds to determine:
- the val distribution is representative of the test distribution
- there isn't a drastic inbalance of gender between train and val splits

In [61]:
df_test['Gender'].value_counts() # 6/11=0.54

Male      6
Female    5
Name: Gender, dtype: int64

In [62]:
train['Gender'].value_counts() # 29/49=0.59

Male      29
Female    20
Name: Gender, dtype: int64

In [63]:
df_val['Gender'].value_counts() # 5/9 = 0.56

Male      5
Female    4
Name: Gender, dtype: int64